In [8]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"
output_file = "../data/evaluation/config5/all_dependencies_all_updated.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)


for entry in data:
    context = entry["context"][:3]
    context_str = "\n\n".join([x["content"] for x in context])
    
    entry["context"] = context
    entry["context_str"] = context_str

with open(output_file, "a", encoding="utf-8") as dest:
    json.dump(data, dest, indent=2)


In [2]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)

    print(len(data))

    print(data[0]["context"][0].keys())

    for x in data:
        context = x["context"]
        if len(context) == 0:
            print(x["index"])

500
dict_keys(['content', 'score', 'index', 'id'])


In [1]:
from prompt_settings import PrompSettingsFactory
from typing import Dict, List
from generator import GeneratorFactory, GeneratorEngine
from dotenv import load_dotenv
from util import load_config
from tqdm import tqdm
import backoff
import json


@backoff.on_exception(backoff.expo, Exception, max_tries=10)
def generate(generator: GeneratorEngine, messages: List) -> str:
    response = generator.generate(messages=messages)

    if not response:
        raise Exception("Response is empty.")

    return response


def run_generation(config: Dict, target_index: List[int]) -> None:
    
    print("Data file: ", config["data_file"])
    print("With RAG: ", config["with_rag"])
    

    with open(config["data_file"], "r", encoding="utf-8") as src:
        data = json.load(src)

    prompt_settings = PrompSettingsFactory.get_prompt_settings(tool_name=config["tool_name"])

    generator = GeneratorFactory().get_generator(
        model_name=config["model_name"], 
        temperature=config["temperature"]
    )

    for entry in tqdm(data, total=len(data), desc="Processing entries"):
        
        if not entry["index"] in target_index:
            continue


        if config["with_rag"]:
            query_str = prompt_settings.query_prompt.format(
                context_str=entry["context_str"], 
                task_str=entry["task_str"],
                format_str=prompt_settings.get_format_prompt()
            )
        else:
            query_str =f"{entry['task_str']}\n\n{prompt_settings.get_format_prompt()}"

        messages = [
            {
                "role": "system", 
                "content": entry["system_str"]
            },
            {
                "role": "user",
                "content": query_str
            }
        ]

        response = generate(
            generator=generator,
            messages=messages
        )

        #entry["responses"] = [response]
        entry["response"] = response
        
    output_file = config["data_file"]
    with open(output_file, "w", encoding="utf-8") as dest:
        json.dump(data, dest, indent=2)
            

config_file = "../generation_config.toml"
env_file = "../.env"

load_dotenv(dotenv_path=env_file)

# load config
config = load_config(config_file=config_file)
run_generation(config=config, target_index=[174, 267, 285])

/home/ssimon/github/cval/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data file:  ../data/evaluation/without/all_dependencies_without_gpt-4o-2024-05-13.json
With RAG:  False


[07/16/24 20:41:05] INFO     Initialize CfgNet prompt settings.                              ]8;id=548139;file:///home/ssimon/github/cval/src/prompt_settings.py\prompt_settings.py]8;;\:]8;id=320272;file:///home/ssimon/github/cval/src/prompt_settings.py#192\192]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=841179;file:///home/ssimon/github/cval/src/generator.py\generator.py]8;;\:]8;id=359751;file:///home/ssimon/github/cval/src/generator.py#47\47]8;;\

Processing entries:   0%|          | 0/500 [00:00<?, ?it/s]

...

Processing entries:  35%|███▌      | 175/500 [00:08<00:16, 19.62it/s]

[07/16/24 20:41:19] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=645900;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=317523;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries:  54%|█████▎    | 268/500 [00:14<00:12, 18.94it/s]

[07/16/24 20:41:27] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=570083;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=795579;file:///home/ssimon/github/cval/env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries: 100%|██████████| 500/500 [00:21<00:00, 23.19it/s]
